In [70]:
import sys
import librosa
from IPython.display import Audio
sys.path.append('../')

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import random

from vocal_patterns.interface.main import predict
from vocal_patterns.ml_logic.preprocessor import scaled_spectrogram, slice_waves, stretch_waveforms
from vocal_patterns.ml_logic.data import get_data, get_manual_test_data
from vocal_patterns.ml_logic.registry import load_model

MLFLOW_TRACKING_URI https://mlflow.lewagon.ai


In [16]:
test_data = get_manual_test_data()
test_data.head()
# test_file = test_data.iloc[10]['path']
# test_file



,path,exercise,filename
0,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,elsa_scale_faster5.7.wav
1,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,Elsa_scale_laptop_2.wav
2,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,Elsa_scale_laptop_1.wav
3,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,Scale_elsa_1.wav
4,/Users/jake/code/jchaselubitz/vocal_patterns/d...,scales,elsa_scale_slower5.3.wav


In [45]:
def plot_spectrogram_librosa(waveform, sr):
    # Compute the spectrogram
    Sxx = librosa.feature.melspectrogram(y=waveform, sr=sr)
    # Convert to dB scale
    Sxx_db = librosa.power_to_db(Sxx, ref=np.max)
    # Plot the spectrogram
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(Sxx_db, sr=sr, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+2.0f dB')
    plt.title('Spectrogram')
    plt.show()

In [5]:
def create_slices(waveform, sr):
 slice_waveforms = slice_waves(waveform, sr) # slices the original one. slice_waveforms is a list of the slices.
# one of the slices (w) is of shape (88200,)

 spectrograms = [] #this will contain my list of slices
 for w in slice_waveforms:
     normalized_spectrogram = scaled_spectrogram(w, sr) # shape = (128, 173, 1)
     spectrograms.append(normalized_spectrogram)
 spectrograms = np.array(spectrograms)
 return spectrograms

In [51]:
# Original
def run_prediction(spectrograms, model):
    raw_predictions = []
    
    for spectrogram in spectrograms:
        spectrogram_expanded = np.expand_dims(spectrogram, axis=0)
    
        prediction = predict(spectrogram_expanded, model) #predict is from main
        raw_predictions.append(prediction)

    # print("raw_predictions_sum", np.mean(raw_predictions, axis=0))
    prediction_map = {
        0: "Arpeggio",
        1: "Other",
        2: "Scale",
    }

    mean_prediction = np.mean(raw_predictions, axis=0)
    prediction = np.argmax(mean_prediction)
    confidence = np.max(mean_prediction) * 100
    prediction_str = prediction_map[prediction]

    return prediction_str, confidence


In [19]:
def noise_up_waveform(y, noise_level=0.1):
    np.random.normal(size=len(y))
    y_noisy = y + (noise_level * np.random.normal(size=len(y)))
    return y_noisy
   

In [20]:
def add_background_noise(waveform, sr, noise_level=0.8):
    sample = "/Users/jake/Desktop/office-ambience-6322.mp3"
    background_sample, sr = librosa.load(sample, sr=sr, mono=True)
    waveform_length = len(waveform)
    sample_length = len(background_sample)
    if sample_length > waveform_length:
        length_difference = sample_length - waveform_length
        # The goal here was to take a random slice of the background sample that is the same length as the waveform
        random_length = random.randint(20, length_difference)
        background_sample = background_sample[random_length:waveform_length+random_length]
        mixed_waveform = waveform + noise_level * background_sample
    return mixed_waveform, sr


In [8]:
test_waveform, sr = librosa.load(test_file, sr=22050)

Audio(test_waveform,rate=22050)

In [74]:
from scripts.recording_test import recording_test

model = load_model(stage="Staging")

recording_test(test_data, model)


Load [Staging] model from MLflow...
✅ Model loaded from MLflow
1/1 [==============================] - 0s 18ms/step
Accuracy: 69.56521739130434 %
Scale Accuracy: 91.66666666666666 %
Arpeggio Accuracy: 45.45454545454545 %


[{'is': 'Scale', 'rslt': 'Scale (95%)', 'name': 'elsa_scale_faster5.7.wav'},
 {'is': 'Scale', 'rslt': 'Scale (71%)', 'name': 'Elsa_scale_laptop_2.wav'},
 {'is': 'Scale', 'rslt': 'Scale (99%)', 'name': 'Elsa_scale_laptop_1.wav'},
 {'is': 'Scale', 'rslt': 'Arpeggio (51%)', 'name': 'Scale_elsa_1.wav'},
 {'is': 'Scale', 'rslt': 'Scale (100%)', 'name': 'elsa_scale_slower5.3.wav'},
 {'is': 'Scale', 'rslt': 'Scale (100%)', 'name': 'elsa_scale_slower5.4.wav'},
 {'is': 'Scale', 'rslt': 'Scale (100%)', 'name': 'elsa_scale_slower5.5.wav'},
 {'is': 'Scale', 'rslt': 'Scale (91%)', 'name': 'elsa_scale_slower5.6.wav'},
 {'is': 'Scale', 'rslt': 'Scale (60%)', 'name': 'elsa_scale_5.2.wav'},
 {'is': 'Scale', 'rslt': 'Scale (87%)', 'name': 'elsa_scale_5.1.wav'},
 {'is': 'Scale', 'rslt': 'Scale (100%)', 'name': 'elsa_scale_faster5.9.wav'},
 {'is': 'Scale', 'rslt': 'Scale (100%)', 'name': 'elsa_scale_faster5.8.wav'},
 {'is': 'Arpeggio', 'rslt': 'Arpeggio (79%)', 'name': 'Arp_elsa_3.wav'},
 {'is': 'Arpeggio

In [10]:
# NOISE
# noise_waveform = noise_up_waveform(orig_waveform, noise_level=0.01)

# plot_spectrogram_librosa(noise_waveform, sr)
# spectrograms = create_slices(noise_waveform, sr)
# prediction_str, confidence = run_prediction(spectrograms)
# print("Prediction:", prediction_str)
# print("Confidence:", confidence)
# Audio(noise_waveform, rate=sr)

In [11]:
# WITH background noise
# w_back_waveform, sr = add_background_noise(test_waveform, sr, noise_level=3)

# plot_spectrogram_librosa(w_back_waveform, sr)
# spectrograms = create_slices(w_back_waveform, sr)
# prediction_str, confidence = run_prediction(spectrograms)
# print("Prediction:", prediction_str)
# print("Confidence:", confidence)
# Audio(w_back_waveform, rate=sr)
